In [12]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, HTML

def solve_wave_equation(L=2, T=2, c=1, Nx=100, Nt=200):
    # Шаги по пространству и времени
    hx = L / (Nx - 1)
    ht = T / (Nt - 1)
    r = c * ht / hx  # Число Куранта
    
    # Проверка устойчивости
    if r > 1:
        print(f"Предупреждение: Число Куранта r={r:.4f} больше 1, схема может быть неустойчива.")
    
    # Сетка
    x = np.linspace(0, L, Nx)
    t = np.linspace(0, T, Nt)
    
    # Начальные условия
    u0 = np.zeros_like(x)  # u(x, 0) = 0
    v0 = np.zeros_like(x)  # u_t(x, 0) = 0
    
    # Граничные условия
    mu0 = lambda t: np.sin(np.pi * (t * c))  # u(0, t) = 0
    muL = lambda t: 0  # u(L, t) = 0
    
    # Источник (правая часть уравнения)
    def f(x, t):
        return np.sin(np.pi * t)  # Источник колебаний
    
    # Аналитическое решение для задачи №4
    def analytical_solution(x, t):
        # Обработка как для скаляра, так и для массива
        is_scalar = np.isscalar(x)
        x_array = np.atleast_1d(x)
        result = np.zeros_like(x_array, dtype=float)
        
        # Функция для решения d'Alembert для точечного источника
        def d_alembert(xi, t):
            if xi < c*t:  # Волна достигла точки xi
                return np.sin(np.pi * (c*t - xi))
            else:  # Волна еще не достигла точки xi
                return 0
        
        # Вычисляем для каждой точки x
        for i in range(len(x_array)):
            result[i] = d_alembert(x_array[i], t)
        
        return result[0] if is_scalar else result
    
    # Инициализация массивов
    u = np.zeros((Nt, Nx))
    u[0, :] = u0  # Начальное условие u(x, 0)
    
    # Первый временной слой с учетом начального условия и источника
    u[1, 1:-1] = (
        u0[1:-1] + ht * v0[1:-1] + 
        (r**2 / 2) * (u0[2:] - 2 * u0[1:-1] + u0[:-2]) + 
        (ht**2 / 2) * f(x[1:-1], 0)
    )
    
    # Применение разностной схемы
    for n in range(1, Nt - 1):
        for i in range(1, Nx - 1):
            # Добавляем источник f(x, t)
            source = ht**2 * f(x[i], t[n])
            u[n + 1, i] = (
                2 * (1 - r**2) * u[n, i]
                - u[n - 1, i]
                + r**2 * (u[n, i + 1] + u[n, i - 1])
                + source
            )
        # Граничные условия
        u[n + 1, 0] = mu0(t[n + 1])
        u[n + 1, -1] = muL(t[n + 1])
    
    return x, t, u, analytical_solution

# Функция для интерактивной визуализации
def plot_results(L=2, T=2, c=1, Nx=100, Nt=200, time_index=100):
    x, t, u, analytical_solution = solve_wave_equation(L, T, c, Nx, Nt)
    
    time = t[time_index]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(x, u[time_index, :], 'b-', linewidth=2, label=f"Численное решение t={time:.2f}")
    
    # Вычисляем аналитическое решение для сравнения
    analytical_values = analytical_solution(x, time)
    ax.plot(x, analytical_values, 'r--', linewidth=1.5, label=f"Аналитическое решение t={time:.2f}")
    
    ax.set_xlabel("x")
    ax.set_ylabel("u(x, t)")
    ax.set_title("Сравнение численного и аналитического решений")
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    # plt.show()
    
    # return fig
    
layout = Layout(width='500px')

interact(
    plot_results,
    L=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='Длина струны (L):', layout=layout),
    T=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='Время (T):', layout=layout),
    c=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description='Скорость волны (c):', layout=layout),
    Nx=IntSlider(min=20, max=200, step=10, value=100, description='Узлы по x:', layout=layout),
    Nt=IntSlider(min=40, max=400, step=20, value=200, description='Узлы по t:', layout=layout),
    time_index=IntSlider(min=0, max=199, value=100, description='Момент времени:', layout=layout)
)

interactive(children=(FloatSlider(value=2.0, description='Длина струны (L):', layout=Layout(width='500px'), ma…

<function __main__.plot_results(L=2, T=2, c=1, Nx=100, Nt=200, time_index=100)>